In [1]:
__author__ = "@Tssp"
__date__   = "17/03/2021"
import sympy as sp
from sympy import diff as D
from sympy import Eq as Eq
from sympy.abc import a, b, c, n
import numpy as np
from utils import chain_rule

In [15]:
def chain_rule(Eq, x, f, new_f, y, relation, order):
    '''
    Parameters
    ----------
    Eq: Equation
    x: independent variable
    f: dependent variable f(x(y))
    new_f: new symbol for the f in string
    y: new independent variable
    relation: equation y(x)
    order: 1 or 2
    
    Returns
    -------
    Equation with the derivatives changed
    '''
    global f_new
    f_new = sp.Function(new_f)(y)
    if order == 2:
        chain = D(f_new, y, order) * D(relation.args[1], x)**2 + D(f_new, y) * D(relation.args[1], x, 2)
        out = Eq.args[0].subs({D(f, x, order): chain}).subs({f: f_new})
        return out
    if order == 1:
        chain = D(f_new, y, order) * D(relation.args[1], x)
        out = Eq.args[0].subs({D(f, x, order): chain}).subs({f: f_new})
        return out

# 1-Particle Hamiltonian

The 1-particle HO Hamiltonian is:
$$\mathcal{H}(r) = -\frac{\hbar^2}{2m}\nabla^2 + \frac{1}{2}m\omega^2r^2$$

In [2]:
x, m, hbar, w, alpha, E = sp.symbols('x m \\hbar \\omega \\alpha E', real=True)
psi = sp.Function('\\psi', real=False)(x)  # wavefunction

In [3]:
Schr_eq = Eq(-hbar**2/(2*m)*D(psi, x, 2) + 0.5*m*w**2*x**2*psi - E*psi, 0)
Schr_eq

Eq(-E*\psi(x) - \hbar**2*Derivative(\psi(x), (x, 2))/(2*m) + 0.5*\omega**2*m*x**2*\psi(x), 0)

In [4]:
Schr_eq_1 = Eq(sp.expand(Schr_eq.args[0] * 2*m/hbar**2), 0)
Schr_eq_1

Eq(-2*E*m*\psi(x)/\hbar**2 - Derivative(\psi(x), (x, 2)) + \omega**2*m**2*x**2*\psi(x)/\hbar**2, 0)

In [5]:
Eq(alpha, sp.sqrt(hbar/(m*w)))

Eq(\alpha, sqrt(\hbar/(\omega*m)))

In [6]:
y = sp.symbols('y')

In [7]:
Schr_eq_2 = Eq(Schr_eq_1.args[0].subs({m*w/hbar: 1/alpha**2}), 0)
Schr_eq_2

Eq(-2*E*m*\psi(x)/\hbar**2 - Derivative(\psi(x), (x, 2)) + x**2*\psi(x)/\alpha**4, 0)

In [8]:
change_xy = Eq(y, x/alpha)
change_xy

Eq(y, x/\alpha)

Let $f(x)$ be a function where x satisfies $x=y\alpha$, $f(x(y\alpha))$ is given by the chain rule:

$$\frac{\partial f(x(y))}{\partial y} = \frac{\partial f}{\partial y}\frac{\partial y}{\partial x}$$

$$\frac{\partial^2 f(x(y))}{\partial y^2} = \frac{\partial^2 f}{\partial y^2}\left(\frac{\partial y}{\partial x}\right)^2 + \frac{\partial f}{\partial y}\frac{\partial^2 y}{\partial x^2}$$

In [16]:
Schr_eq_3 = Eq(chain_rule(Schr_eq_2, x, psi, '\\psi', y, change_xy, 2), 0).subs({x: y*alpha})
Schr_eq_3

Eq(-2*E*m*\psi(y)/\hbar**2 + y**2*\psi(y)/\alpha**2 - Derivative(\psi(y), (y, 2))/\alpha**2, 0)

In [17]:
Schr_eq_4 = Eq(sp.simplify(Schr_eq_3.args[0] * alpha**2), 0).subs({alpha: sp.sqrt(hbar/(m*w))})
Schr_eq_4

Eq(-2*E*\psi(y)/(\hbar*\omega) + y**2*\psi(y) - Derivative(\psi(y), (y, 2)), 0)

In [18]:
eps = sp.symbols('\\varepsilon')
Eq(eps, 2*E/(hbar*w))

Eq(\varepsilon, 2*E/(\hbar*\omega))

In [19]:
Schr_eq_5 = Schr_eq_4.subs({2*E/(hbar*w): eps})
Schr_eq_5

Eq(-\varepsilon*\psi(y) + y**2*\psi(y) - Derivative(\psi(y), (y, 2)), 0)

Proponemos un comportamiento asintótico que decaiga como una gaussiana:

$$\xi(y) = e^{-y^2/2} H(y)$$

Donde $H(y)$ es una función arbitraria

In [20]:
H = sp.Function('H')(y)
Schr_eq_6 = Eq(sp.simplify(Schr_eq_5.subs({f_new: sp.exp(-y**2/2) * H}).doit()).args[0]*sp.exp(y**2/2), 0)
Schr_eq_6

Eq(-\varepsilon*H(y) + 2*y*Derivative(H(y), y) + H(y) - Derivative(H(y), (y, 2)), 0)

In [21]:
-Schr_eq_6.args[0].collect(H)

-2*y*Derivative(H(y), y) - (1 - \varepsilon)*H(y) + Derivative(H(y), (y, 2))

Que es la ecuación de Hermite

In [22]:
Eq(psi, sp.hermite(n, x) * sp.exp(-y**2/2))

Eq(\psi(x), exp(-y**2/2)*hermite(n, x))

In [23]:
Eq(eps, 2*n+1)

Eq(\varepsilon, 2*n + 1)

In [24]:
Eq(2*E/(hbar*w), 2*n+1)

Eq(2*E/(\hbar*\omega), 2*n + 1)

In [25]:
Eq(E, hbar*w*(n+1/2))

Eq(E, \hbar*\omega*(n + 0.5))